In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
road_seg_inr_capac = zload('../temp_files/road_seg_inr_capac.pkz')

In [5]:
# load tmc-day-ave_speed data for AM peak of July
tmc_day_speed_dict_Jul_AM = zload('../temp_files/Jul_AM/tmc_day_speed_dict.pkz')

# load tmc-day-ave_speed data for MD of July
tmc_day_speed_dict_Jul_MD = zload('../temp_files/Jul_MD/tmc_day_speed_dict.pkz')

# load tmc-day-ave_speed data for PM peak of July
tmc_day_speed_dict_Jul_PM = zload('../temp_files/Jul_PM/tmc_day_speed_dict.pkz')

# load tmc-day-ave_speed data for NT of July
tmc_day_speed_dict_Jul_NT = zload('../temp_files/Jul_NT/tmc_day_speed_dict.pkz')

In [6]:
tmc_day_speed_dict_Jul_AM['129N0410818'].ave_speed()

59.444591527987932

In [7]:
tmc_day_capac_flow_minute_dict = {}

for i in range(len(road_seg_inr_capac.tmc)):
    for day in range(32)[1:]:
        tmc = road_seg_inr_capac.tmc[i]
        road_num = road_seg_inr_capac.road_num[i]
        shape_length = road_seg_inr_capac.shape_length[i]
        day = day
        AB_AM_capac = road_seg_inr_capac.AB_AM_capac[i]
        AB_MD_capac = road_seg_inr_capac.AB_MD_capac[i]
        AB_PM_capac = road_seg_inr_capac.AB_PM_capac[i]
        AB_NT_capac = road_seg_inr_capac.AB_NT_capac[i]
        AM_ave_speed = tmc_day_speed_dict_Jul_AM[tmc + str(day)].ave_speed()
        MD_ave_speed = tmc_day_speed_dict_Jul_MD[tmc + str(day)].ave_speed()
        PM_ave_speed = tmc_day_speed_dict_Jul_PM[tmc + str(day)].ave_speed()
        NT_ave_speed = tmc_day_speed_dict_Jul_NT[tmc + str(day)].ave_speed()
        AM_speed_minute = tmc_day_speed_dict_Jul_AM[tmc + str(day)].speed
        MD_speed_minute = tmc_day_speed_dict_Jul_MD[tmc + str(day)].speed
        PM_speed_minute = tmc_day_speed_dict_Jul_PM[tmc + str(day)].speed
        NT_speed_minute = tmc_day_speed_dict_Jul_NT[tmc + str(day)].speed
        tmc_day_capac_flow_minute = RoadSegInrCapacFlowMinute(tmc, road_num, shape_length, day, \
                                                  AB_AM_capac, AB_MD_capac, \
                                                  AB_PM_capac, AB_NT_capac, \
                                                  AM_ave_speed, MD_ave_speed, \
                                                  PM_ave_speed, NT_ave_speed, \
                                                  AM_speed_minute, MD_speed_minute, \
                                                  PM_speed_minute, NT_speed_minute)
        assert(len(tmc_day_capac_flow_minute.AM_flow_minute()) == 120)
        assert(len(tmc_day_capac_flow_minute.MD_flow_minute()) == 120)
        assert(len(tmc_day_capac_flow_minute.PM_flow_minute()) == 120)
        assert(len(tmc_day_capac_flow_minute.NT_flow_minute()) == 120)
        tmc_day_capac_flow_minute_dict[tmc + str(day)] = tmc_day_capac_flow_minute

In [8]:
#zdump(tmc_day_capac_flow_minute_dict, '../temp_files/tmc_day_capac_flow_minute_dict_Jul.pkz')

In [9]:
tmc_day_capac_flow_minute_dict['129N0437720'].NT_flow_minute

<bound method RoadSegInrCapacFlowMinute.NT_flow_minute of <__main__.RoadSegInrCapacFlowMinute object at 0x7fc2b8373690>>

In [10]:
link_with_capac_list = list(zload('../temp_files/links_with_capac.pkz'))

In [11]:
tmc_day_capac_flow_dict = zload('../temp_files/tmc_day_capac_flow_dict_Jul.pkz')

In [12]:
import numpy as np

link_day_minute_Jul_dict = {}

for day in range(32)[1:]:
    i = 0
    for link_with_capac in link_with_capac_list:
        AM_flow = sum([tmc_day_capac_flow_dict[tmc + str(day)].AM_flow() * tmc_length_dict[tmc] / \
                       tmc_day_speed_dict_Jul_AM[tmc + str(day)].ave_speed() \
                       for tmc in link_with_capac.tmc_set]) / \
                  sum([tmc_length_dict[tmc] / tmc_day_speed_dict_Jul_AM[tmc + str(day)].ave_speed() \
                       for tmc in link_with_capac.tmc_set])
        MD_flow = sum([tmc_day_capac_flow_dict[tmc + str(day)].MD_flow() * tmc_length_dict[tmc] / \
                       tmc_day_speed_dict_Jul_MD[tmc + str(day)].ave_speed() \
                       for tmc in link_with_capac.tmc_set]) / \
                  sum([tmc_length_dict[tmc] / tmc_day_speed_dict_Jul_MD[tmc + str(day)].ave_speed() \
                       for tmc in link_with_capac.tmc_set])
        PM_flow = sum([tmc_day_capac_flow_dict[tmc + str(day)].PM_flow() * tmc_length_dict[tmc] / \
                       tmc_day_speed_dict_Jul_PM[tmc + str(day)].ave_speed() \
                       for tmc in link_with_capac.tmc_set]) / \
                  sum([tmc_length_dict[tmc] / tmc_day_speed_dict_Jul_PM[tmc + str(day)].ave_speed() \
                       for tmc in link_with_capac.tmc_set])
        NT_flow = sum([tmc_day_capac_flow_dict[tmc + str(day)].NT_flow() * tmc_length_dict[tmc] / \
                       tmc_day_speed_dict_Jul_NT[tmc + str(day)].ave_speed() \
                       for tmc in link_with_capac.tmc_set]) / \
                  sum([tmc_length_dict[tmc] / tmc_day_speed_dict_Jul_NT[tmc + str(day)].ave_speed() \
                       for tmc in link_with_capac.tmc_set])
        AM_flow_minute = list(sum([np.array(tmc_day_capac_flow_minute_dict[tmc + str(day)].AM_flow_minute()) \
                                   * tmc_length_dict[tmc] / tmc_day_speed_dict_Jul_AM[tmc + str(day)].speed \
                                  for tmc in link_with_capac.tmc_set]) / \
                              sum([np.ones(len(tmc_day_capac_flow_minute_dict[tmc + str(day)].AM_flow_minute())) \
                                   * tmc_length_dict[tmc] / tmc_day_speed_dict_Jul_AM[tmc + str(day)].speed \
                                  for tmc in link_with_capac.tmc_set]))
        MD_flow_minute = list(sum([np.array(tmc_day_capac_flow_minute_dict[tmc + str(day)].MD_flow_minute()) \
                                   * tmc_length_dict[tmc] / tmc_day_speed_dict_Jul_MD[tmc + str(day)].speed \
                                  for tmc in link_with_capac.tmc_set]) / \
                              sum([np.ones(len(tmc_day_capac_flow_minute_dict[tmc + str(day)].MD_flow_minute())) \
                                   * tmc_length_dict[tmc] / tmc_day_speed_dict_Jul_MD[tmc + str(day)].speed \
                                  for tmc in link_with_capac.tmc_set]))
        PM_flow_minute = list(sum([np.array(tmc_day_capac_flow_minute_dict[tmc + str(day)].PM_flow_minute()) \
                                   * tmc_length_dict[tmc] / tmc_day_speed_dict_Jul_PM[tmc + str(day)].speed \
                                  for tmc in link_with_capac.tmc_set]) / \
                              sum([np.ones(len(tmc_day_capac_flow_minute_dict[tmc + str(day)].PM_flow_minute())) \
                                   * tmc_length_dict[tmc] / tmc_day_speed_dict_Jul_PM[tmc + str(day)].speed \
                                  for tmc in link_with_capac.tmc_set]))
        NT_flow_minute = list(sum([np.array(tmc_day_capac_flow_minute_dict[tmc + str(day)].NT_flow_minute()) \
                                   * tmc_length_dict[tmc] / tmc_day_speed_dict_Jul_NT[tmc + str(day)].speed \
                                  for tmc in link_with_capac.tmc_set]) / \
                              sum([np.ones(len(tmc_day_capac_flow_minute_dict[tmc + str(day)].NT_flow_minute())) \
                                   * tmc_length_dict[tmc] / tmc_day_speed_dict_Jul_NT[tmc + str(day)].speed \
                                  for tmc in link_with_capac.tmc_set]))
        link_with_capac_new = Link_with_Free_Flow_Time_Minute(link_with_capac.init_node, link_with_capac.term_node, \
                                                       link_with_capac.tmc_set, \
                                                       link_with_capac.AM_capac, \
                                                       link_with_capac.MD_capac, \
                                                       link_with_capac.PM_capac, \
                                                       link_with_capac.NT_capac, \
                                                       link_with_capac.free_flow_time, \
                                                       link_with_capac.length, \
                                                       AM_flow, MD_flow, PM_flow, NT_flow, \
                                                       AM_flow_minute, MD_flow_minute, \
                                                       PM_flow_minute, NT_flow_minute)
        link_day_minute_Jul_dict['link_' + str(i) + '_' + str(day)] = link_with_capac_new
        i = i + 1

In [13]:
#zdump(link_day_minute_Jul_dict, '../temp_files/link_day_minute_Jul_dict.pkz')

In [14]:
link_day_minute_Jul_dict['link_0_10'].AM_flow

1411.9672832630704

In [15]:
link_day_minute_Jul_dict_JSON = {}
for link_idx in range(24):
    for day in range(32)[1:]:   
        key = 'link_' + str(link_idx) + '_' + str(day)
        data = {'link_idx': link_idx, 'day': day, \
             'init_node': link_day_minute_Jul_dict[key].init_node, \
             'term_node': link_day_minute_Jul_dict[key].term_node, \
             'AM_capac': link_day_minute_Jul_dict[key].AM_capac, \
             'MD_capac': link_day_minute_Jul_dict[key].MD_capac, \
             'PM_capac': link_day_minute_Jul_dict[key].PM_capac, \
             'NT_capac': link_day_minute_Jul_dict[key].NT_capac, \
             'free_flow_time': link_day_minute_Jul_dict[key].free_flow_time, \
             'length': link_day_minute_Jul_dict[key].length, \
             'AM_flow': link_day_minute_Jul_dict[key].AM_flow, \
             'MD_flow': link_day_minute_Jul_dict[key].MD_flow, \
             'PM_flow': link_day_minute_Jul_dict[key].PM_flow, \
             'NT_flow': link_day_minute_Jul_dict[key].NT_flow, \
             'AM_flow_minute': link_day_minute_Jul_dict[key].AM_flow_minute, \
             'MD_flow_minute': link_day_minute_Jul_dict[key].MD_flow_minute, \
             'PM_flow_minute': link_day_minute_Jul_dict[key].PM_flow_minute, \
             'NT_flow_minute': link_day_minute_Jul_dict[key].NT_flow_minute}
        link_day_minute_Jul_dict_JSON[key] = data
        
import json

# Writing JSON data
with open('../temp_files/link_day_minute_Jul_dict_JSON.json', 'w') as json_file:
    json.dump(link_day_minute_Jul_dict_JSON, json_file)

In [16]:
with open('../temp_files/link_day_minute_Jul_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jul_dict_JSON_ = json.load(json_file)

In [17]:
link_day_minute_Jul_dict_JSON_['link_3_9'] ['AM_flow_minute'][0:5]

[6493.728058765033,
 6720.575634412163,
 6796.430475280989,
 6621.581401015268,
 6809.6560460676465]